In [1]:
import os
from dotenv import load_dotenv
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import pandas as pd

load_dotenv()
uri = os.getenv('DATABASE_URI')
client = MongoClient(uri, server_api=ServerApi('1'))
db = client['Instagram']
collection = db["Events"]

# postsDf = pd.read_csv("information_for_mongo.csv")

In [2]:
#time conversion and edit formats
from datetime import datetime
import time

postsDf.reset_index(drop=True)
postsDf = postsDf.fillna(value="None")
postsDf.drop(postsDf.columns[postsDf.columns.str.contains(
    'unnamed', case=False)], axis=1, inplace=True)

for index, row in postsDf.iterrows():
    if row["is_event"] == True:
      row_dict = {}
      row_dict = eval(row["event_details"])
      try:
        conStart = datetime.fromisoformat(row_dict["start_time"])
        unixStart = time.mktime(conStart.timetuple())
        try:
          conEnd = datetime.fromisoformat(row_dict["end_time"])
          unixEnd = time.mktime(conEnd.timetuple())
        except:
          conEnd = None
        row_dict["start_time"]=unixStart
        row_dict["end_time"]=unixEnd
      except:
        postsDf.at[index, "is_event"] = False
        err = f'Start failed {row_dict["start_time"]}'
        print(index, err)
      postsDf.at[index, "event_details"] = row_dict
      print(unixStart,unixEnd)

1725827400.0 1725836400.0
4 Start failed 
1725827400.0 1725836400.0
1725926400.0 1725930000.0
6 Start failed 
1725926400.0 1725930000.0


In [2]:
#resetting db if nexessary
load_dotenv()
uri = os.getenv('DATABASE_URI')
client = MongoClient(uri, server_api=ServerApi('1'))
db = client['Instagram']
collection = db["Events"]

collection.delete_many({})

DeleteResult({'n': 11, 'electionId': ObjectId('7fffffff0000000000000daa'), 'opTime': {'ts': Timestamp(1730593622, 15), 't': 3498}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1730593622, 15), 'signature': {'hash': b'\xee\xad\x89Y\xd6,O\x87Z\x02OQj\x12{\x853\x12\x80T', 'keyId': 7394494294142222396}}, 'operationTime': Timestamp(1730593622, 15)}, acknowledged=True)

In [3]:
from typing import List
from together import Together
import json
import time

together_api_key = os.getenv('TOGETHER_API')


#code for embedding
embedding_model_string = 'WhereIsAI/UAE-Large-V1' # model API string from Together.

def generate_embedding(input_texts: List[str], model_api_string: str) -> List[List[float]]:
  together_client = Together(api_key=together_api_key)
  outputs = together_client.embeddings.create(
      input=input_texts,
      model=model_api_string,
  )
  return [x.embedding for x in outputs.data]

for index, row in postsDf.iterrows():
    if row["is_event"] == True:
      row_dict = {}
      for column in postsDf.columns.tolist():
        if column in ['account','date','caption','accessibility_caption','url','likes','display_photo']:
          row_dict[column] = row[column]
      row_dict["embedded"] = generate_embedding([row["processed_json"]], embedding_model_string)  
      row_dict["event_details"] = row["event_details"]
      result = collection.insert_one(row_dict)
      print(f"Inserted document ID: {result.inserted_id}")
      time.sleep(1)



        

Inserted document ID: 66d9f73dcedbf40b2a3c25f8
Inserted document ID: 66d9f73fcedbf40b2a3c25f9


In [9]:

def return_events(prompt):
    query = prompt
    query_emb = generate_embedding([query], embedding_model_string)[0]

    results = collection.aggregate([
    {
        "$vectorSearch": {
        "queryVector": query_emb,
        "path": "embedded",
        "numCandidates": 100, # this should be 10-20x the limit
        "limit": 2, # the number of documents to return in the results
        "index": "vector_index", # the index name you used in Step 4.
        }
    }
    ])
    response = ""
    for i in results:
        response += (i['caption']+ "\n"+ i['url'] + '\n\n')
    return response

prompt = "Cheese"

print(return_events(prompt))


🔥✨ Join us for our Bonfire End of Term event in collaboration with our friends at CSC, WiCS, and Tech+ ✨🔥

📅 Tuesday, July 23rd
⏰ 6:00 - 9:00 PM
📍Columbia Lake, Fire Pit 1

Please note this event is drop in! No need to stay for the whole time, come and go as you please, open to UWaterloo students😄

Get ready for a fun evening with s'mores, hot dogs (vegan option available), drinks, freezies, and other snacks🍫😋

Don't miss out! See you there 🌐

Sign up using the link in bio🔗
C9i2rTsgxql

Want to gain hands-on experience with product development and improvement? Join us at our upcoming PRODUCT DEVELOPMENT NIGHT!!✨

💼 This event will feature a short lecture about the essentials of product development followed by a collaborative case study facilitated by our very own UW PM execs.

🗓️ Thursday, July 25th
⏰ 6:30 - 8 PM
📍E7 2nd Floor Ideas Clinic (2409)
🥤Free Chungchun for everyone who signs up beforehand😋

Link in bio to sign up :)
C9vfxOhg0hw


